In [ ]:
import pandas as pd
from pathlib import Path
import os

# Tutorial for handling FMU models 
The aim of this tutorial is to demonstrate how to load FMU models on python using <code>ModelicaFmuModel</code> class.

# 1. Proposed model 

In this tutorial, we load and run an FMU model created with Python beforehand.

The model is a uses a resistance-capacity approach (4R2C) of a wall installed in a test bench, with : 
- inside-outside convection/conduction transfers
- Infrared transfers with the sky and surrounding
- Short wave solar radiation transfers
- External and internal temperature as boundary conditions.

The model was built using openModelica then exported as an FMU: 

<img src="images/OM_eticsmodel.png"  height="200">


# 2. Set boundary file
First, let us load measurement data on python, which will be used as our boundary conditions:

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/study_df.csv",
    index_col=0,
    parse_dates=True
)    

# 2. Set simulations options

The used class for running the FMU model requires a model path, simulation options, and optionaly, a reference dataframe for boundary options (to override the default one) and a list of outputs.

We already loaded the boundary file. We can set the simulation options:
- Start time and stop time should be in second. We can use the index of the <code>DataFrame</code> we just created.
The function <code>datetime_to_seconds</code>
helps you convert datetime index in seconds.
- The solver in the simulation options must be one of 'Euler' or 'CVode'.
- The output interval is in seconds.

In [ ]:
import datetime as dt

def datetime_to_seconds(index_datetime):
    time_start = dt.datetime(index_datetime[0].year, 1, 1, tzinfo=dt.timezone.utc)
    new_index = index_datetime.to_frame().diff().squeeze()
    new_index.iloc[0] = dt.timedelta(
        seconds=index_datetime[0].timestamp() - time_start.timestamp()
    )
    sec_dt = [elmt.total_seconds() for elmt in new_index]
    return list(pd.Series(sec_dt).cumsum())

In [ ]:
second_index = datetime_to_seconds(reference_df.index)

In [ ]:
simulation_options_FMU = {
    "startTime":second_index[0],
    "stopTime": second_index[-1],
    "solver": "CVode",  
    "outputInterval": 300,
}

And define a list of output that will be included in the dataframe output for any simulation, here the calculated temperatures between layers of the wall.

In [ ]:
output_list  = [
    "T_coat_ins.T",
     "T_ins_ins.T",
     "Tw_out.T"
]

#  3. Instantiate ModelicaFmuModel

Now, we can also load an FMU <code>ModelicaModel</code> from <code>corrai.fmu</code>: 

Attributes:
- fmu_path: Path to the FMU file.
- simulation_options: A dictionary containing simulation settings such as startTime, stopTime, and stepSize.
- x: Input boundary conditions provided as a pandas.DataFrame.
- output_list: List of simulation output variables.
- simulation_dir: Directory for storing simulation files.
- parameters: Dictionary of simulation parameters.


In [ ]:
from corrai.fmu import ModelicaFmuModel 

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

simu_FMU = ModelicaFmuModel(
    fmu_path=Path(TUTORIAL_DIR) / "resources/etics_v0.fmu",
    simulation_options=simulation_options_FMU,
    x = reference_df,
    output_list=output_list,
)

#  4. Run a simulation

A simulation is run using the <code>simulate()</code> method, with the following parameters:
- parameter_dict (optional): A dictionary containing parameter values for the simulation.
- simulation_options (optional): A dictionary defining simulation-specific settings such as start and stop times or solver types. Here, the simulation options were already provided when instantiating the model.
- x (optional): Boundary condition data as a pandas.DataFrame to be used during the simulation. Already provided.
- solver_duplicated_keep (default: "last"): Handles duplicated solver indices by selecting the desired version ("last" or "first").
- post_process_pipeline (optional): A scikit-learn pipeline to apply post-processing steps on simulation results.
- debug_param (default: False): Prints the parameter_dict if enabled.
- debug_logging (default: False): Enables detailed logging for debugging purposes.
- logger (optional): A custom logger instance for capturing logs during the simulation.


Let's set the initial and parameter values in a dictionary : 
- initial temperatures of internal wall surface, insulation nodes, and coating surface
- value of conductivity of insulation.

In [ ]:
parameter_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
    'Lambda_ins.k': 0.04,
}

And run the simulation:

In [ ]:
init_res_FMU = simu_FMU.simulate(
    parameter_dict = parameter_dict,
    debug_logging=False
)

Results are displayed in a dataframe:

In [ ]:
init_res_FMU

We can quickly plot the results:

In [ ]:
init_res_FMU.plot()